In [13]:
import numpy as np
import pandas as pd
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest
from src.playground_docs import print_bbo
from numba import njit

from numba.typed import List
from numba import int64, float64

import polars as pl

In [2]:
jan_2 = np.load("../data/daily_processed/deribit_eth_perp_2025-01-02.npz")['data']

asset = (BacktestAsset()
        .data([jan_2])
        .linear_asset(1.0)
        .constant_latency(10_000_000, 10_000_000)
        .risk_adverse_queue_model()
        .no_partial_fill_exchange()
        .trading_value_fee_model(0.0002, 0.0007)
        .tick_size(0.1)
        .lot_size(0.001)
)

hbt = HashMapMarketDepthBacktest([asset])

@njit
def plot_bbo(hbt, local_timestamp, best_bid, best_ask):
    while hbt.elapse(1 * 1e9) == 0:
        # Records data points
        local_timestamp.append(hbt.current_timestamp)

        depth = hbt.depth(0)

        best_bid.append(depth.best_bid)
        best_ask.append(depth.best_ask)
    return True

local_timestamp = List.empty_list(int64, allocated=10000)
best_bid = List.empty_list(float64, allocated=10000)
best_ask = List.empty_list(float64, allocated=10000)


plot_bbo(hbt, local_timestamp, best_bid, best_ask)

hbt.close()




0

In [19]:
df = pl.DataFrame({'timestamp': local_timestamp, 'best_bid': best_bid, 'best_ask': best_ask})
df = df.with_columns(
    pl.from_epoch('timestamp', time_unit='ns')
)
df = df[:5000]
plot = df.plot(x="timestamp")

# on the same plot, plot the best bid and best ask lines
plot.line(x='timestamp', y='best_bid', color='blue')
plot.line(x='timestamp', y='best_ask', color='red')


TypeError: 'DataFramePlot' object is not callable